###### imports and installs

In [ ]:
!pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.3/308.3 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
!cd ByteTrack && pip3 install -q -r requirements.txt
!cd ByteTrack && python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric

import sys
sys.path.append(f"/content/ByteTrack")

Cloning into 'ByteTrack'...
remote: Enumerating objects: 2007, done.
remote: Total 2007 (delta 0), reused 0 (delta 0), pack-reused 2007
Receiving objects: 100% (2007/2007), 79.60 MiB | 33.72 MiB/s, done.
Resolving deltas: 100% (1141/1141), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 KB 22.9 MB/s 

In [ ]:
!pip install supervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

import yolox
print("yolox.__version__:", yolox.__version__)

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

import supervision

Ultralytics YOLOv8.0.50 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.9/166.8 GB disk)


yolox.__version__: 0.1.0


In [ ]:
from supervision.draw.color import ColorPalette, DEFAULT_COLOR_PALETTE
from supervision.geometry.core import Point
from supervision.video import VideoInfo
from supervision.video import get_video_frames_generator
from supervision.video import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.detection.core import Detections, BoxAnnotator
from supervision.detection.line_counter import LineZone, LineZoneAnnotator
from supervision.detection.polygon_zone import PolygonZone, PolygonZoneAnnotator

###### storing BYTETracker arguments

In [ ]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
# bytetracker detections need to match those of yolo8 model

from typing import List
import numpy as np

def detection2boxes(detections):
  return np.hstack((detections.xyxy, detections.confidence[:, np.newaxis]))


def tracks2boxes(tracks):
  return np.array([track.tlbr for track in tracks], dtype=float)


def match_detections_with_tracks(detections, tracks):
  if not np.any(detections.xyxy) or len(tracks) == 0:
    return np.empty((0,))
  
  track_boxes = tracks2boxes(tracks)
  iou = box_iou_batch(track_boxes, detections.xyxy)
  track2detection = np.argmax(iou, axis=1)

  tracker_ids = [None] * len(detections)

  for tracker_index, detection_index in enumerate(track2detection):
    if iou[tracker_index, detection_index] != 0:
      tracker_ids[detection_index] = tracks[tracker_index].track_id

  return tracker_ids

###### main

In [ ]:
model = YOLO('yolov8l.pt')
model.fuse()

  0%|          | 0.00/83.7M [00:00<?, ?B/s]

YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs


In [ ]:
video_path = "video_path" # input video 
video_info = VideoInfo.from_video_path(video_path) #Video Info object containing dimensions and number of frame in video
result_path = "result.mp4" # resulting video output
interested_class_ids = [0, 3, 5, 7] # classes we are interested in detecting and tracking

In [ ]:
# extracting the start and end points of the line over which objects must cross to be counted
a = Point(5, int(video_info.height // 1.7)) 
b = Point(int(video_info.width - 5), int(video_info.height // 1.7))


# obtaining Polygon cordinates - Polygons help determine what area of the frame to conduct
# detections and tracking in 
polygon = np.array([[21, 410],[1257, 414],[1261, 682],[13, 682]])

In [ ]:
# initializing necessary objects for tracking, zoning and counting

colors = ColorPalette.from_hex(DEFAULT_COLOR_PALETTE)
byte_tracker = BYTETracker(BYTETrackerArgs)
box_annotator = BoxAnnotator()
line_zone = LineZone(start = a, end = b)
line_zone_annotator = LineZoneAnnotator()
zone = PolygonZone(polygon, frame_resolution_wh=(video_info.width, video_info.height))
zone_annotator = PolygonZoneAnnotator(zone, color = colors.by_idx(1))

In [ ]:
# creating video output while conducting detection, tracking and counting

with VideoSink(result_path, video_info) as video:
  
  for frame in get_video_frames_generator(video_path):

    result = model.predict(frame)

    detections = Detections.from_yolov8(result[0])

    mask = np.array([id in interested_class_ids for id in detections.class_id], dtype = bool)

    detections.filter(mask, inplace=True)

    tracks = byte_tracker.update(output_results = detection2boxes(detections), img_info = frame.shape, img_size = frame.shape)

    track_ids = match_detections_with_tracks(detections, tracks)

    detections.tracker_id = np.array(track_ids)

    second_mask = zone.trigger(detections)
    
    detections = detections[second_mask]

    labels = [f"{tracker_id}-{model.names[class_id]}" for _, _, class_id, tracker_id in detections]

    scene = box_annotator.annotate(scene=frame, detections=detections, labels=labels)

    line_zone_annotator.annotate(frame = scene, line_counter = line_zone)

    line_zone.trigger(detections)

    zone_annotator.annotate(scene)

    video.write_frame(scene)


0: 384x640 6 cars, 3 buss, 3 trucks, 1 bench, 41.8ms
Speed: 1.0ms preprocess, 41.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 3 buss, 3 trucks, 1 bench, 41.1ms
Speed: 0.7ms preprocess, 41.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 cars, 2 buss, 3 trucks, 1 traffic light, 41.2ms
Speed: 0.5ms preprocess, 41.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 6 cars, 1 bus, 4 trucks, 41.2ms
Speed: 0.6ms preprocess, 41.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 7 cars, 1 bus, 4 trucks, 28.4ms
Speed: 1.0ms preprocess, 28.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 1 bus, 4 trucks, 29.6ms
Speed: 0.7ms preprocess, 29.6ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 2 buss, 2 trucks, 28.4ms
Speed: 0.6ms preprocess, 28.4ms inference, 1.7ms postprocess per image 